In [2]:
import shapefile

In [3]:
sf = shapefile.Reader("world/data/VGD-Oesterreich_gen_250/BEV_VGD_250_LAM_mitAttributen_shp_2016_10_02")

In [4]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [5]:
shaperecs = sf.shapeRecords()

In [23]:
from django.contrib.gis.geos import Polygon, MultiPolygon

In [9]:
shaperecs[7].shape.shapeType

5

In [10]:
lst_recs = []
for x in shaperecs:
    if x.shape.shapeType not in lst_recs:
        lst_recs.append(x.shape.shapeType)

In [11]:
lst_recs

[5]

In [62]:
from datetime import date

In [63]:
src = Source(name='BEV Österreich, VGD', original_url="http://www.bev.gv.at/pls/portal/docs/PAGE/BEV_PORTAL_CONTENT_ALLGEMEIN/0200_PRODUKTE/UNENTGELTLICHE_PRODUKTE_DES_BEV/VGD-Oesterreich_gen_250.zip", downloaded=date.today())

In [64]:
src.save()

/home/vagrant/.venvs/geodjango/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1393: RuntimeWarning: DateTimeField Source.downloaded received a naive datetime (2017-01-13 00:00:00) while time zone support is active.
  RuntimeWarning)



In [66]:
sf_fields = sf.fields

In [69]:
sf_fields.pop(0)

('DeletionFlag', 'C', 1, 0)

In [70]:
sf_fields

[['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [109]:
errors_shapes = []

In [110]:
for x in shaperecs:
    try:
        p = Polygon(x.shape.points, srid=31287)
    except:
        errors_shapes.append(x)
        continue
    p.transform(4326)
    p2 = MultiPolygon(p)
    lst_legacy = dict()
    for indx, f in enumerate(sf_fields):
        if type(x.record[indx]) == bytes:
            entry = x.record[indx].decode('latin1')
        else:
            entry = str(x.record[indx])
        lst_legacy[f[0]] = entry
    if type(x.record[1]) == bytes:
        name = x.record[1].decode('latin1')
    else:
        name = x.record[1]
    a = Area(geom=p2, name=name, legacy_properties=lst_legacy, source=src)
    a.save()

GEOS_ERROR: IllegalArgumentException: Points of LinearRing do not form a closed linestring



In [113]:
g = Area.objects.get(pk=4965)

In [114]:
g.legacy_properties['BKZ']

'414'

In [112]:
errors_shapes[0].record

['85103',
 'Matrei in Osttirol Land',
 'M31',
 '70717',
 'Matrei in Osttirol',
 '707',
 'Lienz',
 '082',
 b'Kitzb\xfchel Lienz',
 '850',
 'Lienz',
 '85',
 'Lienz',
 '7',
 'Tirol',
 1,
 b'\xd6sterreich',
 275388233]

### Bezirkgrenzen

In [123]:
sf2 = shapefile.Reader("world/data/stat_oesterreich_pol_bezirke/STATISTIK_AUSTRIA_POLBEZ_20160101.shp")

In [116]:
shaperecs2 = sf2.shapeRecords()

In [124]:
sf2.fields

[('DeletionFlag', 'C', 1, 0), ['ID', 'C', 254, 0], ['NAME', 'C', 254, 0]]

In [125]:
sf2_fields = sf2.fields

In [126]:
sf2_fields.pop(0)

('DeletionFlag', 'C', 1, 0)

In [127]:
sf2_fields

[['ID', 'C', 254, 0], ['NAME', 'C', 254, 0]]

In [129]:
src2 = Source(name='Statistik Österreich, Grenzen politische Bezirke', original_url="https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-politische-bezirke/resource/fc91c2cd-c1e4-41af-b8e9-02615e258cba", downloaded=date.today())

In [130]:
src2.save()

/home/vagrant/.venvs/geodjango/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1393: RuntimeWarning: DateTimeField Source.downloaded received a naive datetime (2017-01-13 00:00:00) while time zone support is active.
  RuntimeWarning)



In [145]:
errors_shapes2 = []

In [146]:
for indx2, x in enumerate(shaperecs2):
    print('import {}'.format(indx2))
    try:
        p = Polygon(x.shape.points, srid=31287)
    except:
        errors_shapes2.append(x)
        shp_closed = x.shape.points
        shp_closed.append(x.shape.points[0])
        p = Polygon(shp_closed, srid=31287)
    p.transform(4326)
    p2 = MultiPolygon(p)
    lst_legacy = dict()
    for indx, f in enumerate(sf2_fields):
        if type(x.record[indx]) == bytes:
            entry = x.record[indx].decode('latin1')
        else:
            entry = str(x.record[indx])
        lst_legacy[f[0]] = entry
    if type(x.record[1]) == bytes:
        name = x.record[1].decode('latin1')
    else:
        name = x.record[1]
    a = Area(geom=p2, name=name, legacy_properties=lst_legacy, source=src2)
    a.save()

import 0
import 1
import 2
import 3
import 4
import 5
import 6
import 7
import 8
import 9
import 10
import 11
import 12
import 13
import 14
import 15
import 16
import 17
import 18
import 19
import 20
import 21
import 22
import 23
import 24
import 25
import 26
import 27
import 28
import 29
import 30
import 31
import 32
import 33
import 34
import 35
import 36
import 37
import 38
import 39
import 40
import 41
import 42
import 43
import 44
import 45
import 46
import 47
import 48
import 49
import 50
import 51
import 52
import 53
import 54
import 55
import 56
import 57
import 58
import 59
import 60
import 61
import 62
import 63
import 64
import 65
import 66
import 67
import 68
import 69
import 70
import 71
import 72
import 73
import 74
import 75
import 76
import 77
import 78
import 79
import 80
import 81
import 82
import 83
import 84
import 85
import 86
import 87
import 88
import 89
import 90
import 91
import 92
import 93
import 94


In [142]:
shaperecs2[0].shape.points

[(637378.7599999998, 439925.47000000067),
 (637377.1099999994, 439927.1400000006),
 (637314.75, 439990.3499999996),
 (637302.6600000001, 440017.9399999995),
 (637288.3100000005, 440050.6799999997),
 (637203.5399999991, 440119.26999999955),
 (637196.8699999992, 440124.6699999999),
 (637191.6999999993, 440128.8499999996),
 (637179.4499999993, 440138.76999999955),
 (637172.8100000005, 440144.1400000006),
 (637160.6500000004, 440154.8499999996),
 (637158.7699999996, 440156.5),
 (637149.1300000008, 440164.98000000045),
 (637122.0099999998, 440188.8499999996),
 (637112.0899999999, 440197.5800000001),
 (637106.2899999991, 440202.6899999995),
 (637098.8200000003, 440209.2599999998),
 (637096.9199999999, 440210.9399999995),
 (637092.9100000001, 440214.48000000045),
 (637089.5700000003, 440217.44999999925),
 (637077.6699999999, 440228.1899999995),
 (637000.0600000005, 440303.47000000067),
 (636986.1699999999, 440318.83999999985),
 (636922.1600000001, 440389.6799999997),
 (636860.9399999995, 4404

### import Austro Hungarian Monarchy 1910

In [21]:
path = 'world/data/reinert/Geodaten/Polygone/censusmosaic/austro-hungarian_empire_1910/Austro-Hungarian Empire 1910/Austro_Hungarian_Empire_1910_v1_0.shp'

In [8]:
import shapefile

In [13]:
sf = shapefile.Reader(path)

In [14]:
shaperecs = sf.shapeRecords()

In [15]:
sf_fields = sf.fields

In [16]:
sf_fields

[('DeletionFlag', 'C', 1, 0),
 ['LAND', 'C', 50, 0],
 ['AUTONUMBER', 'N', 19, 11],
 ['SHAPE_LENG', 'N', 19, 11],
 ['SHAPE_AREA', 'N', 19, 11],
 ['OID_', 'N', 9, 0],
 ['AUTONUMB_1', 'N', 19, 0],
 ['ID_1', 'N', 17, 0],
 ['NAME', 'C', 24, 0],
 ['REGION', 'N', 9, 0],
 ['DISTRICT', 'N', 9, 0],
 ['LOCAL_', 'N', 8, 0]]

In [1]:
from django.contrib.gis.gdal import DataSource
from world.layermapping_fork import LayerMapping

In [22]:
ds = DataSource(path)

In [23]:
layer = ds[0]

In [24]:
print(layer.fields)

['LAND', 'AUTONUMBER', 'SHAPE_LENG', 'SHAPE_AREA', 'OID_', 'AUTONUMB_1', 'ID_1', 'NAME', 'REGION', 'DISTRICT', 'LOCAL_']


In [7]:
print(layer.geom_type)

Polygon


In [8]:
print(layer.srs)

PROJCS["ETRS_1989_LAEA",
    GEOGCS["GCS_ETRS_1989",
        DATUM["European_Terrestrial_Reference_System_1989",
            SPHEROID["GRS_1980",6378137.0,298.257222101]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Lambert_Azimuthal_Equal_Area"],
    PARAMETER["False_Easting",4321000.0],
    PARAMETER["False_Northing",3210000.0],
    PARAMETER["longitude_of_center",10.0],
    PARAMETER["latitude_of_center",52.0],
    UNIT["Meter",1.0]]


In [25]:
mapping = {'name': 'NAME', 'geom': 'POLYGON', 'geonames_id': 'AUTONUMBER'}

In [26]:
lm = LayerMapping(Area, path, mapping)

In [27]:
lm2 = lm.save(verbose=True)

Saved: Hainspach/Hanspach
Saved: Schluckenau/sluknov
Saved: Rumburg/Rumburk
Saved: Warnsdorf
Saved: Friedland/Fridland
Saved: Zwickau/Cvikov
Saved: Boehmisch Kamnitz/ceska
Saved: Rozwadow
Saved: Kratzau/Chrastava
Saved: Reichenberg/Liberec St.
Saved: Tannwald
Saved: Deutsch Gabel/Nemecke Ja
Saved: Haida/Hajda
Saved: Gablonz an der Neisse/Ja
Saved: Tetschen/Decin
Saved: Ulanow
Saved: Bensen/Benesov
Saved: Reichenberg/Liberec
Saved: Zab?otow/Zabo?otiw
Saved: Rochlitz an der Iser/Rok
Saved: Hochstadt an der Iser/Vy
Saved: Marschendorf/Marsov
Saved: Be?z
Saved: Karbitz/Chabarovice
Saved: Schatzlar/Szacler
Saved: Eisenbrod/Szelezny Brod
Saved: Aussig/Usti nad Labem
Saved: Boehmisch Leipa/ceska Li
Saved: Boehmisch Aicha/cesky Du
Saved: Uhnow/Uhniw
Saved: Oberleutensdorf/Horni Li
Saved: Teplitz-Schoenau/Teplice
Saved: Hohenelbe/Vrchlabi
Saved: Dux/Duchcov
Saved: Katharinaberg/Hora Svate
Saved: Niemes/Mimon
Saved: Semil/Semily
Saved: Radziechow/Radechiw
Saved: Tarnobrzeg
Saved: Starkenbach/Jil

In [12]:
lm2

In [28]:
src = Source.objects.get(name='Austro Hungarian Empire 1910 (Reinert)')

In [29]:
for x in lm2:
    if not x.source:
        x.source = src
        x.start_date_written = '1910'
        x.save()

In [11]:
Area.objects.filter(source__isnull=True).delete()

(1711, {'world.Area': 1711})

In [10]:
lm2

[<Area: Hainspach/Hanspach>,
 <Area: Schluckenau/sluknov>,
 <Area: Rumburg/Rumburk>,
 <Area: Warnsdorf>,
 <Area: Friedland/Fridland>,
 <Area: Zwickau/Cvikov>,
 <Area: Boehmisch Kamnitz/ceska>,
 <Area: Rozwadow>,
 <Area: Kratzau/Chrastava>,
 <Area: Reichenberg/Liberec St.>,
 <Area: Tannwald>,
 <Area: Deutsch Gabel/Nemecke Ja>,
 <Area: Haida/Hajda>,
 <Area: Gablonz an der Neisse/Ja>,
 <Area: Tetschen/Decin>,
 <Area: Ulanow>,
 <Area: Bensen/Benesov>,
 <Area: Reichenberg/Liberec>,
 <Area: Zab?otow/Zabo?otiw>,
 <Area: Rochlitz an der Iser/Rok>,
 <Area: Hochstadt an der Iser/Vy>,
 <Area: Marschendorf/Marsov>,
 <Area: Be?z>,
 <Area: Karbitz/Chabarovice>,
 <Area: Schatzlar/Szacler>,
 <Area: Eisenbrod/Szelezny Brod>,
 <Area: Aussig/Usti nad Labem>,
 <Area: Boehmisch Leipa/ceska Li>,
 <Area: Boehmisch Aicha/cesky Du>,
 <Area: Uhnow/Uhniw>,
 <Area: Oberleutensdorf/Horni Li>,
 <Area: Teplitz-Schoenau/Teplice>,
 <Area: Hohenelbe/Vrchlabi>,
 <Area: Dux/Duchcov>,
 <Area: Katharinaberg/Hora Svate>,
 <

In [33]:
ds = DataSource(path, encoding='1252')
layer = ds[0]
for feat in layer:
    print(feat.get('NAME'), feat.get('ID_1'))

Hainspach/Hanspach 301086180
Schluckenau/sluknov 301086181
Rumburg/Rumburk 301082173
Warnsdorf 301102221
Friedland/Fridland 301021048
Zwickau/Cvikov 301017041
Boehmisch Kamnitz/ceska 301099213
Rozwadow 401070158
Kratzau/Chrastava 301079168
Reichenberg/Liberec St. 301080170
Tannwald 301022050
Deutsch Gabel/Nemecke Ja 301017040
Haida/Hajda 301007015
Gablonz an der Neisse/Ja 301022049
Tetschen/Decin 301099214
Ulanow 401045105
Bensen/Benesov 301099212
Reichenberg/Liberec 301079169
Zab?otow/Zabo?otiw 401064148
Rochlitz an der Iser/Rok 301092197
Hochstadt an der Iser/Vy 301092196
Marschendorf/Marsov 301100216
Be?z 401065149
Karbitz/Chabarovice 301002003
Schatzlar/Szacler 301100217
Eisenbrod/Szelezny Brod 301089188
Aussig/Usti nad Labem 301002002
Boehmisch Leipa/ceska Li 301007014
Boehmisch Aicha/cesky Du 301101219
Uhnow/Uhniw 401054124
Oberleutensdorf/Horni Li 301010023
Teplitz-Schoenau/Teplice 301098211
Hohenelbe/Vrchlabi 301024053
Dux/Duchcov 301018043
Katharinaberg/Hora Svate 301010022
Ni

In [4]:
path2 = 'world/data/reinert/Geodaten/Polygone/censusmosaic/german_empire_1871-1945/German Empire 1871-1945/German_Empire_1871_v.1.0.shp'

In [5]:
ds2 = DataSource(path2)

In [6]:
layer2 = ds2[0]

In [7]:
layer2.fields

['AREA', 'PERIMETER', 'ID', 'LAND', 'NAME', 'STATUS', 'RB', 'TYPE']

In [20]:
for feat in layer2:
    print(feat.get('STATUS'), feat.get('ID'))

K 11001
K 11003
K 11005
K 11004
K 11006
S 5101
S 5501
A 5202
S 5502
 5401
S 5201
K 13516
K 13517
K 13525
K 13531
K 13528
K 13506
K 13514
K 13511
K 13529
K 13510
K 13532
K 13519
K 13521
K 13522
K 13518
K 13515
K 13513
K 13507
K 13508
K 13533
K 13523
K 13530
K 13526
K 13527
A 10001
A 10004
A 10003
A 10002
A 10005
S 2020
S 3005
S 2068
S 2075
S 2072
DA 2076
S 2077
S 2079
S 2085
S 2089
RA 2090
DA 2009
S 2022
S 2017
S 2016
S 2059
S 2013
S 2056
S 2029
S 2040
 2006
S 2051
S 2035
S 2032
S 2037
S 2047
S 2091
S 2014
S 2098
S 2092
S 2097
S 2002
S 2015
S 2096
KA 2011
S 2043
DA 3012
S 2053
S 2054
 2023
S 3007
S 3006
S 3016
RA 3017
S 3002
S 3013
S 3011
S 3015
S 2082
S 2046
DA 2001
DA 2101
S 2099
DA 2078
DA 2083
S 2045
DA 2007
DA 2018
DA 2066
RA 2095
DA 2073
 3001
DA 2021
DA 2038
DA 2041
RA 2025
RA 3010
RA 3003
RA 2067
RA 2071
RA 2019
DA 2060
RA 2061
RA 2074
DA 2070
DA 2080
RA 2081
DA 2087
S 2086
DA 2057
RA 2008
DA 2028
RA 2058
RA 2052
RA 2012
RA 2004
RA 2034
DA 2033
RA 2039
S 2005
RA 2031
DA 2030
DA 

In [12]:
src2 = Source.objects.create(name='German Empire 1871-1945 (1871) (Reinert)')

In [14]:
mapping2 = {'name': 'NAME','geonames_id': 'ID', 'geom': 'POLYGON'}

In [15]:
lm = LayerMapping(Area, path2, mapping2)

In [16]:
lm2 = lm.save(verbose=True)

Saved: ZERBST
Saved: DESSAU
Saved: BERNBURG
Saved: KOETHEN
Saved: BALLENSTEDT
Saved: HANSESTADT LUEBECK
Saved: HANSESTADT BREMEN
Saved: CUXHAVEN
Saved: BREMERHAVEN
Saved: HERZOGTUM LAUENBURG
Saved: HANSESTADT HAMBURG
Saved: GROSS-GERAU
Saved: DARMSTADT
Saved: MAINZ
Saved: OPPENHEIM
Saved: WORMS
Saved: GIESSEN
Saved: BUEDINGEN
Saved: NIDDA
Saved: SCHOTTEN
Saved: LAUTERBACH
Saved: NEUSTADT
Saved: BENSHEIM
Saved: HEPPENHEIM
Saved: ERBACH
Saved: DIEBURG
Saved: OFFENBACH
Saved: FRIEDBERG
Saved: GRUENBERG
Saved: ALSFELD
Saved: LINDENFELS
Saved: WIMPFEN
Saved: VILBEL
Saved: BINGEN
Saved: ALZEY
Saved: BRAKE
Saved: BLOMBERG
Saved: DETMOLD
Saved: SCHOETTMAR
Saved: LIPPERODE-CAPPEL
Saved: GREVESMUEHLEN
Saved: STARGARD
Saved: GADEBUSCH
Saved: BOIZENBURG
Saved: WITTENBURG
Saved: HAGENOW
Saved: HAGENOW
Saved: DOEMITZ
Saved: NEUSTADT
Saved: LUEBZ
Saved: PLAU
Saved: DOBERAN
Saved: SEESTADT WISMAR
Saved: NEUBUKOW
Saved: KROEBELIN
Saved: CRIVITZ
Saved: GNOIEN
Saved: BRUEEL
Saved: BUETZOW
Saved: NEUKALEN

In [17]:
for x in lm2:
    x.start_date_written = '1871'
    x.end_date_written = '1871'
    x.save()

In [18]:
for x in lm2:
    x.source = src2
    x.save()

### Test the combination of polygones

In [31]:
from django.contrib.gis.db.models import Union

In [34]:
pol = Area.objects.filter(source=src).aggregate(Union('geom'))

In [35]:
pol

{'geom__union': <Polygon object at 0x7fd24cc19c48>}

In [39]:
Area.objects.create(name='testunion object', geom=MultiPolygon(pol['geom__union']))

<Area: testunion object>

In [41]:
a = Area.objects.get(name='testunion object')

In [42]:
a.source=src
a.save()

<Area: testunion object>

In [43]:
a.pk

19198

### import the europe file

In [4]:
path_europe = 'world/data/reinert/Geodaten/Polygone/censusmosaic/europe_1900-2003/01 Europe Main/Europe_1900_v.1.1.shp'

In [2]:
from django.contrib.gis.gdal import DataSource
from world.layermapping_fork import LayerMapping

In [5]:
ds = DataSource(path_europe)

In [6]:
layer = ds[0]

In [7]:
layer.fields

['AUTONUMBER', 'AREA', 'LEN', 'ID', 'COUNTRY', 'SUBREGION', 'NAME', 'COMMENTS']

In [17]:
for feat in layer:
    print(feat)
    print(feat.get('NAME'), feat.get('COUNTRY'))

Feature FID 0 in Layer<Europe_1900_v.1.1>
ARCHANGELSK 160
Feature FID 1 in Layer<Europe_1900_v.1.1>
FINNMARK CO 130
Feature FID 2 in Layer<Europe_1900_v.1.1>
TROMS COUNT 130
Feature FID 3 in Layer<Europe_1900_v.1.1>
OULUN 60
Feature FID 4 in Layer<Europe_1900_v.1.1>
NORDLAND CO 130
Feature FID 5 in Layer<Europe_1900_v.1.1>
NORRBOTTENS 190
Feature FID 6 in Layer<Europe_1900_v.1.1>
ICELAND 270
Feature FID 7 in Layer<Europe_1900_v.1.1>
VASTERBOTTE 190
Feature FID 8 in Layer<Europe_1900_v.1.1>
NORD-TRONDE 130
Feature FID 9 in Layer<Europe_1900_v.1.1>
JAMTLANDS L 190
Feature FID 10 in Layer<Europe_1900_v.1.1>
VASTERNORR 190
Feature FID 11 in Layer<Europe_1900_v.1.1>
OLONETS 160
Feature FID 12 in Layer<Europe_1900_v.1.1>
SOR-TRONDEL 130
Feature FID 13 in Layer<Europe_1900_v.1.1>
VAASAN 60
Feature FID 14 in Layer<Europe_1900_v.1.1>
KUOPION 60
Feature FID 15 in Layer<Europe_1900_v.1.1>
MORE OGRO 130
Feature FID 16 in Layer<Europe_1900_v.1.1>
VIIPURIN 60
Feature FID 17 in Layer<Europe_1900_v.1.

In [11]:
src = Source.objects.create(name='Europe 1900-2003 (1900) (Reinert)')

In [12]:
mapping = {'name': 'NAME','geonames_id': 'ID', 'geom': 'POLYGON'}

In [13]:
lm = LayerMapping(Area, path_europe, mapping)

In [14]:
lm2 = lm.save()

In [21]:
for x in lm2:
    x.source = src
    x.start_date_written = '1900'
    for z in layer:
        if z.get('ID') == x.geonames_id:
            legacy = dict()
            for zz in layer.fields:
                legacy[zz] = str(z.get(zz))
            x.legacy_properties = legacy
    x.save()

AUTONUMBER
AREA
LEN
ID
COUNTRY
SUBREGION
NAME
COMMENTS
{'SUBREGION': '0', 'AUTONUMBER': '1.0', 'COMMENTS': '', 'NAME': 'ARCHANGELSK', 'ID': '160010', 'COUNTRY': '160', 'AREA': '853356.858534', 'LEN': '31081.621241'}
AUTONUMBER
AREA
LEN
ID
COUNTRY
SUBREGION
NAME
COMMENTS
{'SUBREGION': '0', 'AUTONUMBER': '2.0', 'COMMENTS': '', 'NAME': 'FINNMARK CO', 'ID': '130200', 'COUNTRY': '130', 'AREA': '48683.7942712', 'LEN': '5341.63022796'}
AUTONUMBER
AREA
LEN
ID
COUNTRY
SUBREGION
NAME
COMMENTS
{'SUBREGION': '0', 'AUTONUMBER': '3.0', 'COMMENTS': '', 'NAME': 'TROMS COUNT', 'ID': '130190', 'COUNTRY': '130', 'AREA': '25903.9367289', 'LEN': '4524.05285436'}
AUTONUMBER
AREA
LEN
ID
COUNTRY
SUBREGION
NAME
COMMENTS
{'SUBREGION': '0', 'AUTONUMBER': '4.0', 'COMMENTS': '', 'NAME': 'OULUN', 'ID': '60090', 'COUNTRY': '60', 'AREA': '166761.656894', 'LEN': '4527.81490905'}
AUTONUMBER
AREA
LEN
ID
COUNTRY
SUBREGION
NAME
COMMENTS
{'SUBREGION': '0', 'AUTONUMBER': '5.0', 'COMMENTS': '', 'NAME': 'NORDLAND CO', 'ID': '

In [22]:
from django.contrib.gis.db.models import Union

In [24]:
lst_countries = []
for z in layer:
    if z.get('COUNTRY') not in lst_countries:
        lst_countries.append(z.get('COUNTRY'))

In [26]:
len(lst_countries)

27

In [45]:
for x in lst_countries[10:]:
    pol = Area.objects.filter(source=src, legacy_properties__COUNTRY=str(x)).aggregate(Union('geom'))
    try:
        ar = Area.objects.create(source=src, geom=pol['geom__union'], name=str(x), area_type='Country')
    except:
        ar = Area.objects.create(source=src, geom=MultiPolygon(pol['geom__union']), name=str(x), area_type='Country')
    lst_cnt_com.append(ar)
    for z in Area.objects.filter(source=src, legacy_properties__COUNTRY=str(x)):
        rel = AreaArea.objects.create(related_areaA=ar, related_areaB=z, relation_type='parentCountry')
        lst_rel.append(rel)

In [46]:
lst_countries

[160,
 130,
 60,
 190,
 270,
 170,
 40,
 80,
 50,
 100,
 140,
 120,
 70,
 10,
 20,
 240,
 250,
 200,
 31,
 0,
 110,
 260,
 220,
 181,
 150,
 90,
 91]

In [43]:
lst_cnt_com[9:]

[<Area: 100>]

In [33]:
Area.objects.filter(source=src, legacy_properties__COUNTRY='31')

[<Area: BRABANT>, <Area: LIMBURG>, <Area: LIEGE>, <Area: LUXEMBOUR>, <Area: NAMUR>, <Area: HAINAUT>, <Area: OOST-VLAA>, <Area: WEST-VLAA>, <Area: ANTWERPEN>]

In [36]:
f = Area.objects.get(name='160')

In [37]:
f.pk

19894

In [38]:
f.delete()

(1, {'world.Area': 1, 'world.AreaArea': 0, 'world.Label': 0})

In [41]:
for x in lst_cnt_com:
    print(x.pk)

19895
19896
19897
19898
19899
19900
19901
19902
19903
19904


In [42]:
Area.objects.filter(source=src, area_type='Country').delete

<bound method GeoQuerySet.length of [<Area: 190>, <Area: 60>, <Area: 130>, <Area: 160>, <Area: 100>, <Area: 50>, <Area: 80>, <Area: 40>, <Area: 170>, <Area: 270>]>

In [47]:
rr = AreaArea.objects.filter(related_areaA_id=19920)

In [48]:
rr

[<AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>, <AreaArea: AreaArea object>]

In [50]:
for f in rr:
    print(f.related_areaB.pk)

19792
19798
19799
19806
19810
19815
19816
19819
19820
19822
19823
19824
19825
19877
19878
19879
